In [26]:
import pandas as pd

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType
from pyspark.sql import SparkSession
import os
import sys

In [27]:
os.environ["PYSPARK_PYTHON"]=sys.executable
os.environ["PYSPARK_DRIVER_PATH"] = sys.executable
os.environ["PYSPARK_DRIVER_PATH"]

'd:\\Development\\Processing\\Batch\\Spark\\PySpark\\PysparkPyarrow\\.venv\\Scripts\\python.exe'

In [28]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [29]:
# Declare the function and create the UDF
def multiply_func(a: pd.Series, b: pd.Series) -> pd.Series:
    return a * b

In [30]:
multiply = pandas_udf(multiply_func, returnType=LongType())  # type: ignore[call-overload]

In [31]:
# The function for a pandas_udf should be able to execute with local Pandas data
x = pd.Series([1, 2, 3])
print(multiply_func(x, x))

0    1
1    4
2    9
dtype: int64


In [32]:
# Create a Spark DataFrame, 'spark' is an existing SparkSession
df = spark.createDataFrame(pd.DataFrame(x, columns=["x"])) # type: ignore


In [33]:
# Execute function as a Spark vectorized UDF
df.select(multiply(col("x"), col("x"))).show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "D:\Development\Processing\Batch\Spark\PySpark\PysparkPyarrow\.venv\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1231, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Development\Processing\Batch\Spark\PySpark\PysparkPyarrow\.venv\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1067, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Development\Processing\Batch\Spark\PySpark\PysparkPyarrow\.venv\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 529, in read_single_udf
    f, r